In [1]:
import pandas as pd
import requests
import datetime
from datetime import datetime, timedelta
import os
import geopandas as gpd
from shapely.geometry import Point

In [2]:
app_token = "3qGtNcgvCFOyype97IhPUIMpk"

# 1 Data Processing

## 1.1 Downloading, cleaning, and filtering the 311 Service Request data

In [3]:
api_token = "3qGtNcgvCFOyype97IhPUIMpk"

def date_ranges(start_date, end_date):
    return [(current, (current.replace(day=1) + timedelta(days=32)).replace(day=1) - timedelta(days=1))
            for current in (start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1))
            if current.day == 1]

def fetch_data(start, end, token):
    query = f"created_date between '{start:%Y-%m-%dT%H:%M:%S}' and '{end:%Y-%m-%dT%H:%M:%S}'"
    response = requests.get("https://data.cityofnewyork.us/resource/erm2-nwe9.json",
                            params={"$$app_token": token, "$where": query, "$limit": 999999,
                                    "$select": "unique_key, created_date, closed_date, agency, complaint_type, descriptor, location_type, incident_zip, latitude, longitude, borough"})
    try:
        response.raise_for_status()
        return pd.DataFrame(response.json())
    except requests.RequestException as e:
        print(f"Error during API request: {e}")
        return pd.DataFrame()

def process_csv(df, file_path):
    df.to_csv(file_path, mode='a', index=False, header=not os.path.exists(file_path))

def clean_convert(file_path):
    df = pd.read_csv(file_path, low_memory=False).dropna(subset=['borough'], how='any')
    df[['latitude', 'longitude']] = df[['latitude', 'longitude']].astype(float)
    df['created_date'] = pd.to_datetime(df['created_date'])
    df['closed_date'] = pd.to_datetime(df['closed_date'])
    df['geometry'] = df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
    gdf = gpd.GeoDataFrame(df, geometry='geometry').set_crs(epsg=4326)
    return gdf

start_date, end_date = datetime(2015, 1, 1), datetime(2023, 9, 30)

for start, end in date_ranges(start_date, end_date):
    csv_path = f"data/complaints_data_{start.year}.csv"
    print(f"Downloading data for: {start:%Y-%m-%d} to {end:%Y-%m-%d}")
    process_csv(fetch_data(start, end, api_token), csv_path)

for year in range(start_date.year, end_date.year + 1):
    csv_path = f"data/complaints_data_{year}.csv"
    print(f"Cleaning data for the year: {year}")
    print(clean_convert(csv_path).head())

Cleaning data for the year: 2015


ValueError: could not convert string to float: 'MANHATTAN'

## 1.2 Downloading, cleaning, and filtering the 2015 Tree Cencus data

In [4]:
json_url = "https://data.cityofnewyork.us/api/views/5rq2-4hqu/rows.json?accessType=DOWNLOAD"
app_token = "3qGtNcgvCFOyype97IhPUIMpk"
response = requests.get(json_url, params={"$$app_token": app_token})
data = response.json()["data"]
column_names = [col["name"] for col in response.json()["meta"]["view"]["columns"]]
df = pd.DataFrame(data, columns=column_names)
selected_columns = ['tree_id', 'spc_latin', 'health', 'status', 'zipcode', 'Latitude', 'longitude', 'spc_common']
Tree_Cencus = df[selected_columns]
Tree_Cencus = gpd.GeoDataFrame(Tree_Cencus, geometry=gpd.points_from_xy(Tree_Cencus.longitude, Tree_Cencus.Latitude), crs="EPSG:4326")
Tree_Cencus.crs = "EPSG:4326"
print(Tree_Cencus)
Tree_Cencus.to_csv('data/tree_data.csv',index=False)

       tree_id                           spc_latin health status zipcode  \
0       180683                         Acer rubrum   Fair  Alive   11375   
1       200540                   Quercus palustris   Fair  Alive   11357   
2       204026  Gleditsia triacanthos var. inermis   Good  Alive   11211   
3       204337  Gleditsia triacanthos var. inermis   Good  Alive   11211   
4       189565                     Tilia americana   Good  Alive   11215   
...        ...                                 ...    ...    ...     ...   
683783  155433                   Quercus palustris   Good  Alive   11211   
683784  183795                 Cladrastis kentukea   Good  Alive   11375   
683785  166161                         Acer rubrum   Good  Alive   10314   
683786  184028                         Acer rubrum   Good  Alive   10457   
683787  200607                         Acer rubrum   Fair  Alive   11365   

           Latitude     longitude           spc_common  \
0       40.72309177  -73.8442

## 1.3 Cleaning and filtering the Zip Codes data

In [5]:
def clean_zipcode_data(file):
    gdf = gpd.read_file(file)
    gdf = gdf[['ZIPCODE', 'POPULATION', 'geometry' ]]
    gdf.rename(columns={'ZIPCODE':'zipcode', 'POPULATION':'population', 'geometry':'geometry'}, inplace = True)
    gdf = gdf.reset_index(drop = True)
    return gdf

In [6]:
zipcode = clean_zipcode_data(r"C:\Users\Victo\Desktop\nyc_zipcodes.shp")
print(zipcode.head())
print(zipcode.tail())

  zipcode  population                                           geometry
0   11436     18681.0  POLYGON ((1038098.252 188138.380, 1038141.936 ...
1   11213     62426.0  POLYGON ((1001613.713 186926.440, 1002314.243 ...
2   11212     83866.0  POLYGON ((1011174.276 183696.338, 1011373.584 ...
3   11225     56527.0  POLYGON ((995908.365 183617.613, 996522.848 18...
4   11218     72280.0  POLYGON ((991997.113 176307.496, 992042.798 17...
    zipcode  population                                           geometry
258   10310     25003.0  POLYGON ((950767.507 172848.969, 950787.510 17...
259   11693     11052.0  POLYGON ((1028453.995 167153.410, 1027813.010 ...
260   11249     28481.0  POLYGON ((995877.318 203206.075, 995968.511 20...
261   10162         0.0  POLYGON ((997731.761 219560.922, 997641.948 21...
262   10119         0.0  POLYGON ((986038.661 213051.063, 986135.314 21...


## 1.4 Cleaning and filtering the Zillow data

In [8]:
zillow = pd.read_csv(r'C:\Users\Victo\Desktop\zillow_rent_data.csv')
zillow = zillow[zillow['City'] == 'New York']
selected_columns = ['RegionName', 'CountyName']  # Removed extra square brackets
region_and_county = zillow[selected_columns]  # Removed extra square brackets
time_index = zillow.loc[:, '2018-09-30':]
time_index = time_index.drop(time_index.columns[0], axis=1)
zillow = pd.concat([region_and_county, time_index], axis=1)
zillow = zillow.melt(id_vars=["RegionName", "CountyName"], var_name="date")
zillow = zillow.reset_index(drop=True)
zillow = zillow.dropna()
print(zillow)

      RegionName       CountyName        date        value
0          11385    Queens County  2018-10-31  2419.898399
3          10467     Bronx County  2018-10-31  1774.365406
5          11226     Kings County  2018-10-31  2366.890927
6          11207     Kings County  2018-10-31  2249.035187
7          10025  New York County  2018-10-31  3263.088691
...          ...              ...         ...          ...
8695       10282  New York County  2023-09-30  7347.458333
8696       11109    Queens County  2023-09-30  4529.358974
8697       10006  New York County  2023-09-30  4060.096154
8698       10162  New York County  2023-09-30  5011.666667
8699       10004  New York County  2023-09-30  4355.328283

[5969 rows x 4 columns]


# 2 Storing Data

## 2.1 Creating Database

In [2]:
!createdb LRJY_FINAL_PROJECT
#This is where Lingwen and Joshua both are not able to run
#Lingwen even attempted on his old PC, still not working

^C


In [ ]:
!psql --dbname LRJY_FINAL_PROJECT -c 'CREATE EXTENSION postgis;'

In [ ]:
!psql --dbname LRJY_FINAL_PROJECT -f schema.sql

In [ ]:
import psycopg2

cur = conn.cursor()

## 2.2 Creating Tables

In [ ]:
tree_cencus_table = """
    CREATE TABLE IF NOT EXISTS tree_census (
        id INTEGER PRIMARY KEY,
        geometry geometry,(Geometry, 4326)
        health VARCHAR,
        status VARCHAR,
        zipcode INTEGER NOT NULL,
        species VARCHAR,
        latitude FLOAT,
        longitude FLOAT
        borough VARCHAR,
    );
"""

In [ ]:
nyc_zip_codes_table = """
    CREATE TABLE IF NOT EXISTS nyc_zip_codes (
        id INTEGER PRIMARY KEY,
        zipcode INTEGER NOT NULL,
        population INTEGER,
        geometry geometry(Geometry, 4326)
    );
"""

In [ ]:
zillow_table = """
    CREATE TABLE IF NOT EXISTS zillow_rent (
        id INTEGER PRIMARY KEY,
        zipcode INTEGER NOT NULL,
        county VARCHAR,
        date DATE,
        value FLOAT
    );
"""

In [ ]:
cur.execute(tree_cencus_table)
cur.execute(nyc_zip_codes_table)
cur.execute(zillow_table)
conn.commit()

In [ ]:
conn = psycopg2.connect("dbname=LRJY_FINAL_PROJECT user=postgres password=postgres")
cur = conn.cursor()

for year in range(2015, 2024):
    table_name = f"=311_Service_Request_{year}"
    create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            id INTEGER PRIMARY KEY,
            start_date DATE,
            end_date DATE,
            agency VARCHAR,
            complaint_type VARCHAR,
            descriptor VARCHAR,
            location_type VARCHAR,
            incident_zip INTEGER,
            latitude FLOAT,
            longitude FLOAT,
            borough VARCHAR,
            geometry geometry(Geometry, 4326)
        );
    """
    cur.execute(create_table_query)

conn.commit()